# Using the planner

Source : <https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/05-using-the-planner.ipynb>

In [3]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.14.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.14.1-preview"

Installed Packages Microsoft.SemanticKernel, 1.14.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.14.1-preview

In [4]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

## Setting up Handlebars Planner

In [5]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

## Providing plugins to the planner

In [7]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "prompt_template_samples");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

### Define your Ask

In [8]:
#pragma warning disable SKEXP0060

var ask = """
Tomorrow is Valentine's day.
I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem."
""";

var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 0: Extract Key Values --}}
{{set "significantOther" "your significant other's name"}}
{{set "poemLines" 4}}
{{set "dateIdeas" (array "Idea 1" "Idea 2" "Idea 3" "Idea 4")}}

{{!-- Step 1: Generate Date Ideas Poem --}}
{{#with "dateIdeas" as |ideas|}}
  {{#each (range 0 poemLines)}}
    {{set "lineIndex" @index}}
    {{set "ideaIndex" (subtract lineIndex 1)}}
    {{set "idea" (lookup ideas ideaIndex)}}
    {{set "poemLine" (WriterPlugin-ShortPoem input=idea)}}
    {{json poemLine}}
  {{/each}}
{{/with}}


In [9]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var shakespeareFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings, "Shakespeare");

In [10]:
#pragma warning disable SKEXP0060

var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan);

Updated plan:

{{!-- Step 0: Extract key values --}}
{{set "dateIdeas" (array "Thou art like a star, let us go afar, and watch the night sky together" "In fair Verona, let us roam, and recreate the love of Romeo and Juliet" "By the riverside, let us sit, and read sonnets of love and wit" "In Shakespearean attire, let us dance, and enjoy a night of Renaissance")}}

{{!-- Step 1: Generate the poem with date ideas --}}
{{#each dateIdeas}}
  {{set "poemLine" (concat "For our date on Valentine's day, " this)}}
  {{set "poem" (concat poem poemLine "\n")}}
{{/each}}

{{!-- Step 2: Print the poem --}}
{{json (concat "On Valentine's day, thou shalt have these date ideas:\n" poem)}}


In [11]:
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Original Plan results:

There once was a cat named Fred,
Who got stuck in a loaf of bread.
He wiggled and squirmed,
But his
tummy was firm,
Now he&#39;s a breaded cat instead!
    
    
    
    
    There once was a robot
named Zero,
Who loved to dance the tango.
But oh, what a sight,
When he danced with all his might,
He tripped on his wires and yelled &quot;Oh no!&quot;
    
    
    
    
    There once was a cat
named Fred,
Who got stuck in a loaf of bread.
He wiggled and squirmed,
But his tummy was firm,
Now
he&#39;s a breaded cat instead!
    
    
    
    
    There once was a cat named Fred,
Who got
stuck in a loaf of bread.
He wiggled and squirmed,
But his tummy was firm,
Now he&#39;s a breaded
cat instead!
    
    
    
    
    There once was a cat named Fred,
Who got stuck in a loaf of
bread.
He wiggled and squirmed,
But his tummy was firm,
Now he&#39;s a breaded cat instead!



In [18]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("New Plan results:\n");
foreach(var line in newPlanResult.ToString().Split($"\\n"))
{
    Console.WriteLine(line);
}

New Plan results:

On Valentine's day, thou shalt have these date ideas:
For our date on Valentine's day, Thou art like a star, let us go afar, and watch the night sky together
For our date on Valentine's day, In fair Verona, let us roam, and recreate the love of Romeo and Juliet
For our date on Valentine's day, By the riverside, let us sit, and read sonnets of love and wit
For our date on Valentine's day, In Shakespearean attire, let us dance, and enjoy a night of Renaissance

